In [1]:
import spacy
from spacy.tokenizer import Tokenizer
import nltk.data
import nltk
from nltk import word_tokenize
import re
from nltk import FreqDist
import pandas as pd
import re
import csv
import numpy as np
import csv
import itertools
import os
import pickle
import glob
import nltk
# Classe de cores para destacar certas partes dos prints

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Carregando os dados coletados em páginas Web

Os dados são informações textuais salvas em planilhas do tipo .csv

In [23]:
# Criando uma lista com todas as listas existentes no diretório passado como parâmetro
DATA_PATH = "../dados/datasetParaRotulacaoManual"
DATA_PATH_2 = "../dados/DadosColetados/*.csv"
files_Lists = glob.glob (DATA_PATH_2)
print(color.BOLD+'Quantidade de planilhas:'+color.END, len(files_Lists))

Quantidade de planilhas: 24


In [24]:
# Definindo uma lista que comportará todos os dataframes existentes no diretorio passado com parâmetro na linha de código acima

dataFrames_Lists = []

### Criando dataframes a partir das planilhas .csv que contém os dados coletados

In [25]:
# Função que transformando as planilhas .csv em dataframes. E inserindo cada dataframe em uma posição da 'lista_De_DataFrames' 
# OBS: Executar esse comando apenas uma vez, pois a cada execução os dados são duplicados.

def cria_DataFrames(files_Lists, dataFrames_Lists):    
    #percorrendo a lista 'file' que contém os caminhos de cada planilha
    for f in files_Lists:
        csv = pd.read_csv(f)  # com index_col=2, faz com que o dataframe incie da coluna com id igual a 2
        #adiciona o dataframe a lista de dataframes.
        dataFrames_Lists.append(csv) 
    print(color.BOLD +'Quantidade de datasets:'+color.END,len(dataFrames_Lists))

### Removendo colunas não necessárias dos dataframes

In [26]:
#Função que remover colunas. Algumas colunas como a 'Unnamed: 0' não possuem informações relevantes

def removeColuna(dataFrames_Lists):
    for dataFrame in dataFrames_Lists:
        dataFrame = dataFrame.drop(columns = ['Unnamed: 0','_id','DATA_COLETA', 'LINK'], inplace=True)  

### Inserindo nas celulas os nomes das colunas dos dataframes

Alguns dados coletados não possuem informações textuais suficientes para contextualizá-lo, por exemplo; no atributo graduação alcoólica alguns dados consitem apenas em uma numeração ou númeração seguida de percentual; no atributo local de produção em alguns casos temos apenas o nome do local onde a bebida foi produtiza; no atributo volume temos apenas números seguidos da siglas ml e vol, e da palavra volume.
Para resolver este problema de dados sem contexto texto, optou-se por inserir textos que descrevem o que esses dados represetam, por exemplo, nos dados relacionados a quantidade de alcoól presente na bebida, inserimos o texto que representa sua categoria de entidade nomeada "Graduação alcoólica" (ex: Graduação alcoólica: 35%).
Alguns textos já iniciam com termos que os contextualizam, nestes casos não é necessário inserir nenhuma informação, apenas manter sua forma original.

In [27]:
#'listaTermos', possui os termos de contextualização que já vem no inicio dos dados coletados.
listaTermos = ['Volume', 'Graduação Alcoólica', 'Tipo', 'Local de Produção', 'Grad. Alcool', 'Envelhecimento', 'Harmonização', 'Tempo de Envelhecimento','Produzida em', 'Local', 'Grad. Alcoólica', 'Modo de Produção', 'Barril', 'Graduação alcoólica', 'Palavras comumente usadas', 'Produtor', 'Madeira', 'Grad Alcool','Origem', 'paladar', 'aparência', 'harmonização', 'aroma', 'Graduação Alcóolica', 'Região','Coloração', 'Graduação alcoólica', 'Graduação','VISÃO','PALADAR','OLFATO']

#Essa função verifica se os dados dos atributos já possuem algum dos termos contidos na 'listaTermos'.
#Caso o dado possua algum do termos, etnão nada é alterado, mas caso não possua nenhum dos termo, o nome da coluna que representa
#o atributo em questão é inserido no inicio do dado.
def verificaPalavras(verificaTermo, coluna):   
    for termo in listaTermos:
        if verificaTermo.startswith(termo + ":"):
            #print("Aqui")
            return verificaTermo
    return coluna.replace('_',' ').lstrip()+": " +verificaTermo.lstrip()

In [28]:
#Função que percorre cada dataframe da lista de dataframes e, concatena o nome da coluna ao texto de cada celula dos dataframes

def insere_NomesColunas(dataFrames_Lists):
    for index, dataFrame in enumerate(dataFrames_Lists):
        
        #cria uma lista com os nomes das colunas do dataframe
        list_of_columns = dataFrames_Lists[index].columns.tolist()
        
        #percorre a lista com nomes das colunas, seleciona a coluna do dataframe com o item da lista em questão e...
        #concatena todos os dados da referida coluna com o item da lista referente ao nome da coluna.
        for column in list_of_columns:
            dataFrames_Lists[index][column] = dataFrames_Lists[index][column].apply(lambda x: f"{verificaPalavras(str(x),column)}") 
            #dataFrames_Lists[index][column] = dataFrames_Lists[index][column].apply(lambda x: f"{verificaPalavras(str(x),column).lower()}") 

### Separando os textos por ponto de continuação

In [29]:
#Função que separa por '.' (ponto de continuação) os textos das colunas que possuem textos maiores.

def separando_Texto(dataFrames_Lists):
    for index, dataFrame in enumerate(dataFrames_Lists):
        if "DESCRIÇÃO_DA_CACHAÇA" in dataFrame:
            dataFrames_Lists[index]["DESCRIÇÃO_DA_CACHAÇA"] = dataFrames_Lists[index]["DESCRIÇÃO_DA_CACHAÇA"].apply(lambda x: nltk.sent_tokenize(x))
        if 'HISTÓRIA_DO_ALAMBIQUE' in dataFrame:
            dataFrames_Lists[index]["HISTÓRIA_DO_ALAMBIQUE"] = dataFrames_Lists[index]["HISTÓRIA_DO_ALAMBIQUE"].apply(lambda x: nltk.sent_tokenize(x))
        if 'ANÁLISE_DO_CACHACIER' in dataFrame:
            dataFrames_Lists[index]["ANÁLISE_DO_CACHACIER"] = dataFrames_Lists[index]["ANÁLISE_DO_CACHACIER"].apply(lambda x: nltk.sent_tokenize(x))

### Chamando todas as funções

In [30]:
#Chamada das funções criadas
cria_DataFrames(files_Lists, dataFrames_Lists)
removeColuna(dataFrames_Lists)
insere_NomesColunas(dataFrames_Lists)
separando_Texto(dataFrames_Lists)

Quantidade de datasets: 24


In [31]:
#verifica quantas instâncias existem em cada dataset que representa cada site de venda de bebidas. 
soma=0
for item in dataFrames_Lists:
    tamanho=len(item)
    print(len(item))
    soma=soma+tamanho
print(soma)

576
111
61
31
13
164
90
299
1080
4
183
139
98
15
10
5
69
6
227
170
4
8
12
6
3381


### Criação de um único dataset com todos os dados coletados

In [32]:
dataset_base = pd.concat(dataFrames_Lists, ignore_index=True, sort=False)
dataset_base

,NOME_DA_CACHAÇA,PREÇO,DESCRIÇÃO_DA_CACHAÇA,VOLUME,GRADUAÇÃO_ALCOÓLICA,RECIPIENTE_DE_ARMAZENAMENTO,TIPO,PRODUTOR,LOCAL_DE_PRODUÇÃO,TEMPO_DE_ARMAZENAMENTO,...,HARMONIZAÇÃO,PREMIOS,ANÁLISE_DO_CACHACIER,INFORMAÇÕES_DO_PRODUTO,MODO_DE_PRODUÇÃO,PALAVRAS_COMUMENTE_USADAS,INGREDIENTES,INFORMAÇÕESOES_TÉCNICAS,TEXTO_DESCRITIVO,CONSUMO
0,NOME DA CACHAÇA: Cachaça Três Coronéis,"PREÇO: R$ 35,00",[DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 7...,Volume: 750ml,"Graduação Alcoólica: 39,2%",RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Carv...,Tipo: Envelhecida,Produtor: Glauri Ind Com. Ltda,Local de Produção: Boituva / São Paulo,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NOME DA CACHAÇA: Cachaça Princesa Isabel Jaque...,"PREÇO: R$ 85,00",[DESCRIÇÃO DA CACHAÇA: Na beira do Rio Doce cu...,Volume: 500ml,Graduação Alcoólica: 40%,RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Jaqu...,Tipo: Armazenada,Produtor: Princesa Isabel,Local de Produção: Linhares / Espírito Santo,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NOME DA CACHAÇA: Cachaça Colombina Chita,"PREÇO: R$ 100,00",[DESCRIÇÃO DA CACHAÇA: Colombina Chita 700ml],Volume: 700ml,Graduação Alcoólica: 40%,RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Jatobá,Tipo: Envelhecida,Produtor: RXM Agropecuária,Local de Produção: Alvinopolis / MG,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NOME DA CACHAÇA: Cachaça colombina 3 anos,"PREÇO: R$ 52,00",[DESCRIÇÃO DA CACHAÇA: Envelhecida em seculare...,Volume: 700ml,"Graduação Alcoólica: 41,5%",RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Paró...,Tipo: Armazenada,Produtor: RXM Agropecuaria,Local de Produção: Alvinopolis / MG,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NOME DA CACHAÇA: Cachaça Colombina 10 anos,"PREÇO: R$ 272,00",[DESCRIÇÃO DA CACHAÇA: Cachaça Colombina 10 an...,Volume: 700ml,Graduação Alcoólica: 45%,RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: 10 a...,Tipo: Armazenada,Produtor: RXM Agropecuaria,Local de Produção: Alvinopolis / MG,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3376,NOME DA CACHAÇA: WiBA! Blend de Carvalhos Prem...,"PREÇO: R$80,00","[DESCRIÇÃO DA CACHAÇA: Essa WIBA!, Blend de Ca...",VOLUME: 670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PREMIOS: Medalha de Ouro 2017 na Categoria Pre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3377,NOME DA CACHAÇA: WIBA! Blend de Carvalhos 670ml,"PREÇO: R$63,00","[DESCRIÇÃO DA CACHAÇA: Esta WIBA!, é o resulta...",VOLUME: 670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PREMIOS: Eleita por especialistas uma das 50 m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3378,NOME DA CACHAÇA: WIBA! Senses Premium 750ml,"PREÇO: R$190,00","[DESCRIÇÃO DA CACHAÇA: WIBA!, Senses., Quando ...",VOLUME: 750.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PREMIOS: Medalha de Prata no Concurso Mundial ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3379,NOME DA CACHAÇA: WIBA! Amburana 670ml,"PREÇO: R$59,00","[DESCRIÇÃO DA CACHAÇA: Esta WIBA!, é a tradici...",VOLUME: 670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PREMIOS: Medalha de Ouro no 17º concurso de Vi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#Verifica os atributos que o dataset_base possui
for col in dataset_base.columns:
    print(col)

NOME_DA_CACHAÇA
PREÇO
DESCRIÇÃO_DA_CACHAÇA
VOLUME
GRADUAÇÃO_ALCOÓLICA
RECIPIENTE_DE_ARMAZENAMENTO
TIPO
PRODUTOR
LOCAL_DE_PRODUÇÃO
TEMPO_DE_ARMAZENAMENTO
ANÁLISE_SENSORIAL
DESCRIÇÃO_DA_CACHACA
HISTÓRIA_DO_ALAMBIQUE
TIPO_DO_PRODUTO
HARMONIZAÇÃO
PREMIOS
ANÁLISE_DO_CACHACIER
INFORMAÇÕES_DO_PRODUTO
MODO_DE_PRODUÇÃO
PALAVRAS_COMUMENTE_USADAS
INGREDIENTES
INFORMAÇÕESOES_TÉCNICAS
TEXTO_DESCRITIVO
CONSUMO


### Criação do dataset para rotulação manual
Este dataset será utilizado para rotulação manual dos dados e suas sentenças, serão utilizadas no datset_dados_rotulados(dataset rotulado para treinamento dos modelos de REN)

In [34]:
#Seleção de quantos dados serão selecionados para compor o dataset com 1000 instâncias, isto é, o dataset para rotulação manual
#Os dataframes 2,3,4,5,9,13,14,15,17,29,21,22 e 23 não foram modificados porque possuem a quantidade necessaria de dados para compor o dataset de 1000 instâncias

dataFrames_Lists[0] = dataFrames_Lists[0].iloc[0:69]
dataFrames_Lists[5] = dataFrames_Lists[5].iloc[0:69]
dataFrames_Lists[7] = dataFrames_Lists[7].iloc[0:69]
dataFrames_Lists[8] = dataFrames_Lists[8].iloc[0:69]
dataFrames_Lists[10] = dataFrames_Lists[10].iloc[0:69]
dataFrames_Lists[11] = dataFrames_Lists[11].iloc[0:69]
dataFrames_Lists[16] = dataFrames_Lists[16].iloc[0:69]
dataFrames_Lists[18] = dataFrames_Lists[18].iloc[0:69]
dataFrames_Lists[19] = dataFrames_Lists[19].iloc[0:69]

dataFrames_Lists[1] = dataFrames_Lists[1].iloc[0:68]
dataFrames_Lists[6] = dataFrames_Lists[6].iloc[0:68]
dataFrames_Lists[12] = dataFrames_Lists[12].iloc[0:68]

In [35]:
#verifica se foram selecionadas as quantidades corretas de dados em cada datase individual.
soma = 0
for index, item in enumerate(dataFrames_Lists):
    print('Planilha:',index+1, '->','Quantidade de dados:', len(item))
    soma = soma+ len(item)
print('Quantidade total de dados coletados:',soma)

Planilha: 1 -> Quantidade de dados: 69
Planilha: 2 -> Quantidade de dados: 68
Planilha: 3 -> Quantidade de dados: 61
Planilha: 4 -> Quantidade de dados: 31
Planilha: 5 -> Quantidade de dados: 13
Planilha: 6 -> Quantidade de dados: 69
Planilha: 7 -> Quantidade de dados: 68
Planilha: 8 -> Quantidade de dados: 69
Planilha: 9 -> Quantidade de dados: 69
Planilha: 10 -> Quantidade de dados: 4
Planilha: 11 -> Quantidade de dados: 69
Planilha: 12 -> Quantidade de dados: 69
Planilha: 13 -> Quantidade de dados: 68
Planilha: 14 -> Quantidade de dados: 15
Planilha: 15 -> Quantidade de dados: 10
Planilha: 16 -> Quantidade de dados: 5
Planilha: 17 -> Quantidade de dados: 69
Planilha: 18 -> Quantidade de dados: 6
Planilha: 19 -> Quantidade de dados: 69
Planilha: 20 -> Quantidade de dados: 69
Planilha: 21 -> Quantidade de dados: 4
Planilha: 22 -> Quantidade de dados: 8
Planilha: 23 -> Quantidade de dados: 12
Planilha: 24 -> Quantidade de dados: 6
Quantidade total de dados coletados: 1000


### Salvando os dados 
Nesta parte do código também é feito algumas atividade de pré-processamento, por exemplo, remoção de quebras de linhas.
Também inserimos o id, para identificar onde uma instância inicia.

In [43]:
with open("./datasetParaRotulacaoManual/dados_Para_RotulacaoManual.txt", "w", encoding='utf-8') as file_object:
    count = 0
    for indexPosicaoDataFrame, dataFrame in enumerate(dataFrames_Lists):

        for index, row in dataFrame.iterrows():
            #Insere o id referente a cada instância
            file_object.write('id: '+str(count) + '\n')
            print(count)
            count +=1
            #variavel_id = 'id: '+ str(indexPosicaoDataFrame) + '_' + str(index) + '\n'
            #file_object.write(variavel_id)
            row = row.tolist() # transofrma o serie.series em list
            
            for i in range(0, len(row)): # percorre os elementos de cada linha
                if not re.search(': nan', str(row[i])):
                    if type(row[i]) is list:
                        # atribui a variavel text cada elemento da lista 'row' na posição i,
                        variavel = ''.join(str(x).strip() + '\n' for x in row[i])#.rstrip() 

                    else:
                        variavel = str(row[i])+'\n'
                        #list_Of_Datas.append(row[i]) 
                        
                    variavel = re.sub(r'\r', "", variavel) #removendo todos os \r
                    variavel = re.sub(r'\t', "", variavel) #removendo todos os \t(tabulação)
                    variavel = re.sub(r' +', " ", variavel) #removendo mais de um white space juntos
                    variavel = re.sub(r"\xa0", " ", variavel)
                    variavel = re.sub(r"\s\n", "\n", variavel)
                    variavel = re.sub(r"\n{2,}", "\n", variavel)
                    file_object.write(variavel.lstrip())
                    print(repr(variavel.lstrip())) #PRINT PARA VER CADA LINHA SENDO INSERIDA NO .txt

0
'NOME DA CACHAÇA: Cachaça Três Coronéis\n'
'PREÇO: R$ 35,00\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 700ml\n'
'Volume: 750ml\n'
'Graduação Alcoólica: 39,2%\n'
'RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Carvalho\n'
'Tipo: Envelhecida\n'
'Produtor: Glauri Ind Com. Ltda\n'
'Local de Produção: Boituva / São Paulo\n'
1
'NOME DA CACHAÇA: Cachaça Princesa Isabel Jaqueira Ouro\n'
'PREÇO: R$ 85,00\n'
'DESCRIÇÃO DA CACHAÇA: Na beira do Rio Doce cultivamos cacau na forma mais tradicional e sustentável.\nLá, um certo dia no ano de 2015, uma frondosa jaqueira secular tombou durante um temporal, foi quando nosso patriarca teve essa grande ideia.\nRecolheu o tronco caído e levou para que nosso tanoeiro fizesse barris com a madeira.\nDe volta à fazenda Tupã eles foram enchidos de cachaça Princesa Isabel e o resultado foi essa surpreendente e singular bebida.\nUma bela cachaça, com coloração amarelho esverdeado, límpida, brilhante a muito encorpada.\nNo olfato percebemos pouco álcool, destacan

66
'NOME DA CACHAÇA: Cachaça de Rolha\n'
'PREÇO: R$ 32,00\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça de Rolha armazenada em inox\n'
'Volume: 700ml\n'
'Graduação Alcoólica: 41%\n'
'Local de Produção: São Gonçalo do Amarante / Ceará\n'
67
'NOME DA CACHAÇA: Dom Bre Amburana\n'
'PREÇO: R$ 120,00\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça armazenada em tonéis de Amburana Armazenada em tonéis de amburana por um longo tempo, selecionada e elaborada harmoniosamente, com notas amadeiradas, frutadas e florais, com sabor e aroma exclusivos de uma bebida singular.\nA Dom Bré Amburana foi eleita em 9° Lugar na categoria Envelhecida entre as 50 melhores do Ano no IV Ranking da Cúpula da Cachaça de 2020.\n'
'Volume: 700ml\n'
'Graduação Alcoólica: 40%\n'
'Local de Produção: Guarani / Minas Gerais\n'
68
'NOME DA CACHAÇA: Dom Bre Extra Premium Carvalho\n'
'PREÇO: R$ 126,00\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça armazenada em tonéis de carvalho Bebida equilibrada onde o amadeirado em sintonia com o frutado e abaunilhado result

'DESCRIÇÃO DA CACHAÇA: CACHAÇA SELETA PORCELANA 670 ML\nCachaça Seleta Porcelana é produzida na capital mundial da Cachaça  Salinas / MG descansa em toneis de Umburana por 2,5 anos até ser envasada nas garrafas\nA cachaça é uma das mais belas expressões da cultura do nosso país.\nA Seleta, maior produtora de cachaça artesanal do Brasil, se inspira na alegria, tradição e diversidade dos brasileiros, para produzir uma bebida que seja a imagem do bom gosto e exigência do nosso povo.\nA cachaça artesanal mais vendida do país.\nFeita a partir de canas rigorosamente selecionadas é armazenadas em tonéis de Umburana — planta conhecida por suas características curativas e digestivas.\nA Seleta tem sabor suave e delicioso.\nAparência:Levemente dourada, límpida e brilhante.\nAroma:Possui aromas intensos: amadeirados, frutados, adocicados, com notas de baunilha.\nSabor:Seu sabor é levemente adocicado com notas finais de baunilha, amêndoas e canela.\nHarmonização:A cachaça é uma bebida famosa por c

'DESCRIÇÃO DA CACHAÇA: 150 ml.\n40% em volume.\nEnvelhecida 4 anos em barril de carvalho.\nCachaça Companheira Prata de Bolso – A cachaça de aroma delicado e sabor suave, própria para paladares exigentes.\n'
'GRADUAÇÃO ALCOÓLICA: 40% em volume.\n'
'PREMIOS: Medalha de prata no Concurso de Degustação às Cegas 2014, realizado durante a Expocachaça, em Belo Horizonte, Minas Gerais, Brasil.\nMedalha de bronze no Concurso de Degustação às Cegas 2012, realizado durante a Expocachaça Dose Dupla no Mercado Municipal de São Paulo.\n'
242
'NOME DA CACHAÇA: Cachaça Gramiais\n'
'PREÇO: R$ 29,90\n'
'Volume: 600 ml\n'
'DESCRIÇÃO DA CACHAÇA: Grad.\nAlcool.\n: 42,% Vol.)\nEnvelhecida em barril de Balsamo\nTempo de Envelhecimento: 3 Anos\nConteúdo : 600 ml\nGarrafa : tradicional\nPrazo de Validade : Indeterminado\nEvite o consumo Excessivo de Álcool\n'
'Tempo de Envelhecimento: 3 anos\n'
243
'NOME DA CACHAÇA: Cachaça Seleta 270 ml\n'
'PREÇO: R$ 24,90\n'
'Volume: 270 ml\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça

'Volume: 600 ml\n'
'GRADUAÇÃO ALCOÓLICA: 42%\n'
'RECIPIENTE DE ARMAZENAMENTO: Amendoim Envelhecimento: 6 meses\n'
'LOCAL DE PRODUÇÃO: Lençóis Paulista / SP\n'
324
'NOME DA CACHAÇA: Cachaça João Mendes Ouro Carvalho 700 ML\n'
'PREÇO: R$51,90\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça João Mendes Ouro Carvalho 700 ML ORIGEM: Cachaça Mineira produzida em Perdões - Minas Gerais -Brasil Produzida no Sítio Vó Zirica por João Batista Mendes Cachaça de alambique destilada do mosto fermentado de caldo da cana-de-açúcar Não contém glúten.\nValidade Indeterminada.\nMadeira: Envelhecida de 2 a 4 anos em tonéis de carvalho.\nConteúdo: 700ml - Garrafa Cachaça do Brasil/Brazil Graduação Alcoólica: 38% Medidas: Altura: 28, 5 cm - Diâmetro: 7,2 cm MAPA: MG 005811 0001-9 Para refinar mais seu sabor, a cachaça é envelhecida em tonéis de carvalho, por um período entre dois e quatro anos enriquecendo ainda mais seu aroma.\nO resultado é uma cachaça artesanal com verdadeiro sabor mineiro.\n'
'Volume: 700 ML\n'
'GRAD

...

'DESCRIÇÃO DA CACHAÇA: Cachaça Santo Grau Itirapuã 750 ml.\nA Santo Grau Itirapuã é uma cachaça clássica tradicional do interior paulista.\nCom aroma suave tem sabor aveludado, macio e amplo na boca.\nA cana é moída pela força da roda da água, movimentada por um riacho que, durante anos, fazia todo o engenho funcionar.\n'
'Volume: 750 ml\n'
'GRADUAÇÃO ALCOÓLICA: 41% vol.\n'
703
'NOME DA CACHAÇA: Cachaça Santo Grau Séc XVIII\n'
'PREÇO: R$ 419,90\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça Santo Grau Séc XVIII 998 ml.\nFruto de uma das melhores safras do engenho de Coronel Xavier Chaves nas últimas décadas, a Cachaça Santo Grau Séc XVIII foi produzida em 2002 passou por processo de envelhecimento peculiar.\nIniciou, no ano de 2002, seu armazenamento em tanques de pedra, passando para uma adega climatizada naturalmente, construída há mais de 100 anos.\nMedalha de prata na edição Spirits Selection by Concours Mondial de Bruxelles, no Chile em 2017.\n'
'Volume: 998 ml\n'
'GRADUAÇÃO ALCOÓLICA: 40% vol

'VISÃO: incolor\nOLFATO: frutado\nPALADAR: adocicado, frutado\n'
797
'NOME DA CACHAÇA: Cachaça Princesa Isabel Jaqueira 500ml\n'
'PREÇO: R$ 83,90\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça Princesa Isabel Jaqueira 500ml\nEspírito Santo - Brasil\nConheça mais a  Cachaça Princesa Isabel Jaqueira\nCachaça Princesa Isabel Jaqueira é uma cachaça premiada!\nO lema da Cachaça Princesa Isabel é: Paixão não tem muita explicação vem de dentro vem do peito.\nHarmonize ao seu gosto!\nSeguindo este ideal tudo que é feito com paixão é bem feito!\nPara Adão e Pedro produtores da Princesa Isabel.\nA Cachaça traz o Brasil em seu DNA.\nCada conquista que a Cachaça obteve nacionalmente e internacionalmente deve-se honrar os antepassados que lutaram para passar as tradições da produção do nosso destilado para a próxima geração.\nA Cachaça hoje é algo de festividades de recepção feita para pessoas.\nCachaça é muito mais que uma bebida 100% nacional é um símbolo.\nEspecialmente para promover momentos se divertir apr

'GRADUAÇÃO ALCOÓLICA: 34%\n'
852
'NOME DA CACHAÇA: Cachaça Blend Nordestino\n'
'PREÇO: R$ 161,90\n'
'VOLUME: 750ml\n'
'DESCRIÇÃO DA CACHAÇA: 750ml\n8%\nprata\nSão Sebastião/AL\n'
'INFORMAÇÕESOES TÉCNICAS: Volume\nGraduação Alcoólica\nTipo\nOrigem\n'
'TEXTO DESCRITIVO: Teor Alcoólico: 45,8%\nVolume: 750ml\nTipo: Prata\nOrigem: São Sebastião/AL\n'
'GRADUAÇÃO ALCOÓLICA: 8%\n'
'TIPO: prata\n'
'LOCAL DE PRODUÇÃO: São Sebastião/AL\n'
853
'NOME DA CACHAÇA: Cachaça Maria Izabel Tradicional 700ml\n'
'PREÇO: R$ 172,90\n'
'VOLUME: 700ml\n'
'DESCRIÇÃO DA CACHAÇA: 700ml\n44%\nParaty/RJ\n'
'INFORMAÇÕESOES TÉCNICAS: Volume\nGraduação Alcoólica\nOrigem\n'
'TEXTO DESCRITIVO: Descansada em Inox\nGraduação Alcoólica: 44%\nOrigem: Paraty/RJ\numa cachaça divina, com pouca acidez, onde se percebe claramente a qualidade da bebida.\n'
'GRADUAÇÃO ALCOÓLICA: 44%\n'
'LOCAL DE PRODUÇÃO: Paraty/RJ\n'
854
'NOME DA CACHAÇA: Cachaça Engenho São Luiz Premium Carvalho 600ml\n'
'PREÇO: R$ 76,90\n'
'VOLUME: 600ml\n'
'DES

'DESCRIÇÃO DA CACHAÇA: 700ml\n40%\nblend de madeiras (castanheira e carvalho)\n4 anos\nextra premium\nAmerico Brasiliense/SP\n'
'INFORMAÇÕESOES TÉCNICAS: Volume\nGraduação Alcoólica\nMadeira\nTempo de Armazenamento\nTipo\nOrigem\n'
'Madeira: Blend de Madeiras (castanheira e Carvalho)\nEnvelhecimento: 4 Anos\nTeor Alcoólico: 40%\nVolume: 750ml\nTipo: Extra Premium\nOrigem: Americo Brasiliense/SP\n'
'GRADUAÇÃO ALCOÓLICA: 40%\n'
'RECIPIENTE DE ARMAZENAMENTO: blend de madeiras (castanheira e carvalho)\n'
'TEMPO DE ARMAZENAMENTO: 4 anos\n'
'TIPO: extra premium\n'
'LOCAL DE PRODUÇÃO: Americo Brasiliense/SP\n'
892
'NOME DA CACHAÇA: Cachaça Charmosa de Minas Ouro 980 ML\n'
'PREÇO: R$ 41,90\n'
'VOLUME: 980ml\n'
'DESCRIÇÃO DA CACHAÇA: 980ml\n42%\ncerejeira e garapa amarela\n2 anos\nouro\nCarangola/MG\n'
'INFORMAÇÕESOES TÉCNICAS: Volume\nGraduação Alcoólica\nMadeira\nTempo de Armazenamento\nTipo\nOrigem\n'
'TEXTO DESCRITIVO: A Cachaça Charmosa de Minas é produzida no município de Carangola-MG. El

'NOME DA CACHAÇA: Cachaça Terra de Ouro balsamo\n'
'PREÇO: R$ 27,50\n'
'DESCRIÇÃO DA CACHAÇA: Cachaça Terra de Ouro balsamo 300ml A Cachaça Terra de Ouro é produzida na cidade mineira de Salinas, região internacionalmente reconhecida por abrigar as melhores cachaças existentes no mercado mundial.\nSua elaboração é totalmente artesanal, dotada de segredos e técnicas de gente que vive em meio à produção de cachaça desde que nasceu, conservando a essência das cachaças de Salinas.Madeira: BálsamoGrad.\nAlcoólica: 42%\u200bVolume: 300 mlTempo de Envelhecimento: 2 anosOrigem: Salinas/MGColoração: DouradaAnálise: Sabor suave.\nAroma misto de madeira (bálsamo) com a pura cachaça artesanal.\n'
'Volume: 300ml\n'
951
'NOME DA CACHAÇA: Cachaça Terra de Ouro prata\n'
'PREÇO: R$ 27,50\n'
'DESCRIÇÃO DA CACHAÇA: A Cachaça Terra de Ouro é produzida na cidade mineira de Salinas, região internacionalmente reconhecida por abrigar as melhores cachaças existentes no mercado mundial.\nSua elaboração é totalm

In [45]:
#Step 1 - Lê os dados salvos linha por linha como .txt (código acima). 
#Step 2 - Salva cada linha em uma lista
#Step 3 - Salva a lista como .bit usando a função pickle

list_Of_Datas = []
with open("./datasetParaRotulacaoManual/dados_Para_RotulacaoManual.txt", "r",encoding='utf-8') as fp:
    list_Lines = fp.readlines()
    for line in list_Lines:
        list_Of_Datas.append(line.replace('\n',''))  
        
with open("./datasetParaRotulacaoAutomatica/dados_Para_RotulacaoAutomatica.bin", "wb") as fp:
    pickle.dump(list_Of_Datas,fp)
    
#Verifica se os dados .bin estão corretos
with open("./datasetParaRotulacaoAutomatica/dados_Para_RotulacaoAutomatica.bin", "rb") as fp:
    teste = pickle.load(fp)
teste

['id: 0',
 'NOME DA CACHAÇA: Cachaça Três Coronéis',
 'PREÇO: R$ 35,00',
 'DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 700ml',
 'Volume: 750ml',
 'Graduação Alcoólica: 39,2%',
 'RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Carvalho',
 'Tipo: Envelhecida',
 'Produtor: Glauri Ind Com. Ltda',
 'Local de Produção: Boituva / São Paulo',
 'id: 1',
 'NOME DA CACHAÇA: Cachaça Princesa Isabel Jaqueira Ouro',
 'PREÇO: R$ 85,00',
 'DESCRIÇÃO DA CACHAÇA: Na beira do Rio Doce cultivamos cacau na forma mais tradicional e sustentável.',
 'Lá, um certo dia no ano de 2015, uma frondosa jaqueira secular tombou durante um temporal, foi quando nosso patriarca teve essa grande ideia.',
 'Recolheu o tronco caído e levou para que nosso tanoeiro fizesse barris com a madeira.',
 'De volta à fazenda Tupã eles foram enchidos de cachaça Princesa Isabel e o resultado foi essa surpreendente e singular bebida.',
 'Uma bela cachaça, com coloração amarelho esverdeado, límpida, brilhante a muito encorpada.',
 'No olfato p